In [1]:
from pymysql import connect
from pandas import DataFrame
from numpy import zeros, int64, float64, multiply, dot, sum
from itertools import permutations

In [2]:
db_name = input()

mondial


In [3]:
db = db_name
connection = connect(host="localhost", user="root", password="", db=db)
cursor = connection.cursor()

db_setup = db_name + "_setup"
connection_setup = connect(host="localhost", user="root", password="", db=db_setup)
cursor_setup = connection_setup.cursor()

db_bn = db_name + "_bn"
connection_bn = connect(host="localhost", user="root", password="", db=db_bn)
cursor_bn = connection_bn.cursor()

In [4]:
entities = {}
relations = {}
keys = {}

In [5]:
cursor_setup.execute("SELECT TABLE_NAME FROM entitytables");
entity_tables = cursor_setup.fetchall()
for i in entity_tables:
    cursor.execute("SELECT * FROM " + i[0])
    rows = cursor.fetchall()
    cursor.execute("SHOW COLUMNS FROM " + db + "." + i[0])
    columns = cursor.fetchall()
    entities[i[0]] = DataFrame(rows, columns=[columns[j][0] for j in range(len(columns))])
    cursor_setup.execute("SELECT COLUMN_NAME FROM entitytables WHERE TABLE_NAME = " + "'" + i[0] + "'")
    key = cursor_setup.fetchall()
    keys[i[0]] = key[0][0]

In [6]:
cursor_setup.execute("SELECT TABLE_NAME FROM relationtables")
relation_tables = cursor_setup.fetchall()
for i in relation_tables:
    cursor.execute("SELECT * FROM " + i[0])
    rows = cursor.fetchall()
    cursor.execute("SHOW COLUMNS FROM " + db + "." + i[0])
    columns = cursor.fetchall()
    relations[i[0]] = DataFrame(rows, columns=[columns[j][0] for j in range(len(columns))])
    cursor_setup.execute("SELECT COLUMN_NAME FROM foreignkeycolumns WHERE TABLE_NAME = " + "'" + i[0] + "'")
    key = cursor_setup.fetchall()
    keys[i[0]] = key[0][0], key[1][0]

In [7]:
relation_names = tuple(i[0] for i in relation_tables)

In [8]:
indices = {}
for i in entity_tables:
    cursor_setup.execute("SELECT COLUMN_NAME FROM entitytables WHERE TABLE_NAME = '" + i[0] + "'")
    key = cursor_setup.fetchall()[0][0]
    indices[key] = {}
    for index, row in entities[i[0]].iterrows():
        indices[key][row[key]] = index

In [9]:
matrices = {}
for i in relation_tables:
    cursor_setup.execute("SELECT REFERENCED_TABLE_NAME FROM foreignkeycolumns WHERE TABLE_NAME = " + "'" + i[0] + "'")
    reference = cursor_setup.fetchall()
    matrices[i[0]] = zeros((len(entities[reference[0][0]].index), len(entities[reference[1][0]].index)))

In [10]:
for i in relation_tables:
    cursor_setup.execute("SELECT COLUMN_NAME FROM foreignkeycolumns WHERE TABLE_NAME = '" + i[0] + "'")
    key = cursor_setup.fetchall()
    cursor_setup.execute("SELECT COLUMN_NAME, REFERENCED_COLUMN_NAME FROM foreignkeycolumns WHERE TABLE_NAME = '" + i[0] + "'")
    reference = cursor_setup.fetchall()
    for index, row in relations[i[0]].iterrows():
        matrices[i[0]][indices[reference[0][1]][row[key[0][0]]]][indices[reference[1][1]][row[key[1][0]]]] = 1

In [11]:
cursor_setup.execute("SELECT COLUMN_NAME, TABLE_NAME FROM attributecolumns")
attribute_columns = cursor_setup.fetchall()
attributes = {}
for i in attribute_columns:
    attributes[i[0]] = i[1]

In [12]:
cursor_bn.execute("SELECT DISTINCT child FROM final_path_bayesnets_view")
child = cursor_bn.fetchall()
for i in range(len(child)):
    rule = [child[i][0]]
    cursor_bn.execute("SELECT parent FROM final_path_bayesnets_view WHERE child = " + "'" + child[i][0] + "'")
    parents = cursor_bn.fetchall()
    for j in parents:
        if j[0] != '':
            rule += [j[0]]
    print(rule)
    print('Local_Mults:')
    cursor_bn.execute("SELECT * FROM `" + child[i][0] + "_cp`")
    values = cursor_bn.fetchall()
    multiple = 1
    if len(rule) == 1:
        multiple = 0
    relation_check = 0
    for j in rule:
        if j.find(',') != -1:
            relation_check = 1
    for j in values:
        unmasked_matrices = []
        unmasked_variables = []
        relations_count = 0
        for k in range(len(rule)):
            functor = rule[k].split('(')[0]
            if rule[k].find(',') == -1:
                variable = rule[k].split('(')[1][:-1]
                entity = variable[:-1]
                if relation_check == 0:
                    unmasked_matrices.append(zeros((len(entities[entity].index), 1)))
                    unmasked_variables.append(variable)
                    for l in range(len(entities[entity][functor])):
                        value = entities[entity][functor][l]
                        if type(j[k+multiple]) == str:
                            if type(value) == int64:
                                value = str(value)
                            elif type(value) == float64:
                                value = str(int(value))
                        if value == j[k+multiple]:
                            unmasked_matrices[k + relations_count][indices[keys[entity]][entities[entity][keys[entity]][l]]][0] = 1
                else:
                    for l in rule:
                        l_functor = l.split('(')[0]
                        if l_functor not in relation_names:
                            l_functor = attributes[l_functor]
                        if l.find(',') != -1 and l.find(variable) != -1:
                            unmasked_matrices.append(zeros(matrices[l_functor].shape))
                            unmasked_variables.append(l.split('(')[1][:-1])
                            for m in range(len(entities[entity][functor])):
                                if entities[entity][functor][m] == j[k+multiple]:
                                    if variable == l.split('(')[1].split(',')[0]:
                                        unmasked_matrices[k + relations_count][indices[keys[entity]][entities[entity][keys[entity]][m]],:] = 1
                                    elif variable == l.split('(')[1].split(',')[1][:-1]:
                                        unmasked_matrices[k + relations_count][:,indices[keys[entity]][entities[entity][keys[entity]][m]]] = 1
                            relations_count += 1
                    relations_count -= 1
            else:
                unmasked_variables.append(rule[k].split('(')[1][:-1])
                if functor in relation_names:
                    if j[k+multiple] == 'F':
                        unmasked_matrices.append(1 - matrices[functor])
                        unmasked_matrices[k + relations_count] = 1 - matrices[functor]
                    else:
                        unmasked_matrices.append(matrices[functor])
                        unmasked_matrices[k + relations_count] = matrices[functor]
                else:
                    relation = attributes[functor]
                    if j[k+multiple] == 'N/A':
                        unmasked_matrices.append(1 - matrices[relation])
                        unmasked_matrices[k + relations_count] = 1 - matrices[relation]
                    else:
                        unmasked_matrices.append(zeros(matrices[relation].shape))
                        for l in range(len(relations[relation][functor])):
                            if relations[relation][functor][l] == j[k+multiple]:
                                unmasked_matrices[k + relations_count][indices[keys[relation][0]][relations[relation][keys[relation][0]][l]]][indices[keys[relation][1]][relations[relation][keys[relation][1]][l]]] = 1 
        masked_matrices = [unmasked_matrices[0]]
        masked_variables = [unmasked_variables[0]]
        for k in range(1, len(unmasked_matrices)):
            mask_check = 0
            for l in range(len(masked_matrices)):
                if unmasked_variables[k] == unmasked_variables[l]:
                    masked_matrices[l] = multiply(masked_matrices[l], unmasked_matrices[k])
                    mask_check = 1
            if mask_check == 0:
                masked_matrices.append(unmasked_matrices[k])
                masked_variables.append(unmasked_variables[k])
        sorted_matrices = []
        sorted_variables = []
        if relation_check == 0:
            sorted_matrices.append(masked_matrices[0])
            sorted_variables.append(masked_variables[0])
        else:
            matrices_permutations = list(permutations(masked_matrices))
            variables_permutations = list(permutations(masked_variables))
            for k in range(len(variables_permutations)):
                matrices_chain = []
                variables_chain = []
                first = variables_permutations[k][0].split(',')[0]
                second = variables_permutations[k][0].split(',')[1]
                matrices_chain.append(matrices_permutations[k][0])
                variables_chain.append(variables_permutations[k][0])
                untransposed_check = 1
                transposed_check = 1
                if len(variables_permutations[k]) > 1:
                    for l in range(1, len(variables_permutations[k])):
                        next_first = variables_permutations[k][l].split(',')[0]
                        next_second = variables_permutations[k][l].split(',')[1]
                        if second == next_first:
                            second = next_second
                            matrices_chain.append(matrices_permutations[k][l])
                            variables_chain.append(next_first + ',' + next_second)
                        elif second == next_second:
                            second = next_first
                            matrices_chain.append(matrices_permutations[k][l].T)
                            variables_chain.append(next_second + ',' + next_first)    
                        else:
                            untransposed_check = 0
                            break
                    if untransposed_check != 1:
                        matrices_chain[0] = matrices_permutations[k][0].T
                        variables_chain[0] = second + ',' + first
                        temp = first
                        first = second
                        second = temp
                        for k in range(1, len(variables_permutations[k])):
                            next_first = variables_permutations[k][l].split(',')[0]
                            next_second = variables_permutations[k][l].split(',')[1]
                            if second == next_first:
                                second = next_second
                                matrices_chain.append(matrices_permutations[k][l])
                                variables_chain.append(next_first + ',' + next_second)
                            elif second == next_second:
                                second = next_first
                                matrices_chain.append(matrices_permutations[k][l].T)
                                variables_chain.append(next_second + ',' + next_first)    
                            else:
                                transposed_check = 0
                                break
                if untransposed_check == 1 or transposed_check == 1 or len(variables_permutations[k]) == 1:
                    sorted_matrices = matrices_chain
                    sorted_variables = variables_chain
                    break
        stacked_matrices = [sorted_matrices[0]]
        stacked_variables = [sorted_variables[0]]
        for k in range(1, len(sorted_variables)):
            stacked_matrices.append(sorted_matrices[k])
            stacked_variables.append(sorted_variables[k])
            second = sorted_variables[k].split(',')[1]
            for l in range(k - 1, -1, -1):
                previous_first = sorted_variables[l].split(',')[0]
                if previous_first == second:
                    ring = sorted_matrices[l]
                    stacked_variables.pop(l)
                    stacked_matrices.pop(l)
                    for m in range(l+1, k+1):
                        ring = dot(ring, sorted_matrices[m])
                        stacked_variables.pop(l)
                        stacked_matrices.pop(l)
                    for m in range(ring.shape[0]):
                        for n in range(ring.shape[1]):
                            if m != n:
                                ring[m][n] = 0
                    stacked_variables.append(previous_first + ',' + second)
                    stacked_matrices.append(ring)
        result = stacked_matrices[0]
        for k in range(1, len(stacked_matrices)):
            result = dot(result, stacked_matrices[k])
        print(sum(result))
    print('---------------------------------------------------------------------------------------------------------------')

['agricul(eco0)', 'ecor(country0,eco0)', 'ecor(country1,eco0)', 'service(eco0)']
Local_Mults:
136162.0
169280.0
203505.0
203505.0
238099.0
368.0
920.0
920.0
920.0
736.0
368.0
920.0
920.0
920.0
736.0
2.0
5.0
5.0
5.0
4.0
101568.0
33856.0
169280.0
236992.0
372416.0
552.0
184.0
920.0
1288.0
2024.0
552.0
184.0
920.0
1288.0
2024.0
3.0
1.0
5.0
7.0
11.0
101937.0
101568.0
270848.0
304704.0
169280.0
368.0
552.0
1472.0
1656.0
920.0
368.0
552.0
1472.0
1656.0
920.0
2.0
3.0
8.0
9.0
5.0
169649.0
305442.0
236992.0
135424.0
101568.0
736.0
1288.0
1288.0
736.0
552.0
736.0
1288.0
1288.0
736.0
552.0
4.0
7.0
7.0
4.0
3.0
406641.0
338929.0
33856.0
33856.0
2024.0
1656.0
184.0
184.0
2024.0
1656.0
184.0
184.0
11.0
9.0
1.0
1.0
---------------------------------------------------------------------------------------------------------------
['border(country0,country1)', 'continent(country1)', 'percentage(country0)']
Local_Mults:
1656.0
1656.0
1691.0
2600.0
773.0
1476.0
1462.0
1471.0
2250.0
673.0
1393.0
1401.0
1378.0
